In [1]:
from openai import OpenAI
import json
import httpx
import subprocess
import os
import shutil  
import pandas as pd

In [9]:
import subprocess

# 切割45分钟的视频，每10分钟一个片段
input_video = 'input.mp4'
split_duration = 600  # 10分钟转为秒

# 分五段处理，最后一段包括剩余时间
for i in range(5):
    start_time = split_duration * i
    output_file = f'prepart{i+1}.mp4'
    if i == 4:  # 对于最后一段，直到视频结束
        subprocess.run(['ffmpeg', '-i', input_video, '-ss', str(start_time), '-c', 'copy', output_file], check=True)
    else:
        subprocess.run(['ffmpeg', '-i', input_video, '-ss', str(start_time), '-t', str(split_duration), '-c', 'copy', output_file], check=True)

    # 对每段视频进行压缩和调整分辨率
    compressed_output = f'part{i+1}.mp4'
    subprocess.run(['ffmpeg', '-i', output_file, '-b:v', '50k', '-bufsize', '100k', '-vf', 'scale=-2:180', compressed_output], check=True)


In [8]:
# 转录音频文件成文本
client = OpenAI()

# 遍历5个音频文件
for i in range(1, 6):
    # 构造文件名
    file_name = f"part{i}.mp4"
    # 打开文件
    with open(file_name, "rb") as audio_file:
        # 调用OpenAI API进行音频转录
        transcription = client.audio.transcriptions.create(
            model="whisper-1", 
            file=audio_file,
            language="zh",
            response_format="verbose_json",
            timestamp_granularities=["segment"]
        )

        # 提取特定字段
        extracted_data = [
            {"id": item["id"], "start": item["start"], "end": item["end"], "text": item["text"]} 
            for item in transcription.segments
        ]

        # 将提取的数据保存为JSON文件
        json_file_path = f'Part{i}_audio2text.json'
        with open(json_file_path, 'w', encoding='utf-8') as f:
            json.dump(extracted_data, f, ensure_ascii=False, indent=4)

In [2]:
#将所有JSON文件合并为一个CSV文件

import pandas as pd
import json
import os

# 创建空的数据帧用于存储所有数据
all_data = pd.DataFrame()

# 遍历1到5的文件名
for i in range(1, 6):
    json_file_path = f'Part{i}_audio2text.json'
    # 检查文件是否存在
    if os.path.exists(json_file_path):
        # 读取JSON文件
        with open(json_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            # 转换为DataFrame
            df = pd.DataFrame(data)
            # 添加'name'和'check'列
            df['name'] = f'prepart{i}.mp4'
            df['check'] = 0
            # 合并到总数据帧
            all_data = pd.concat([all_data, df], ignore_index=True)

# 保存到CSV文件
all_data.to_csv('text.csv', index=False)
